<img src="../images/logo.jpg" style="width:85px;height:85px;float:left" /><h1 style="position:relative;float:left;display:inline">Writing a GAN: The Estimator way</h1>

<a href='https://colab.research.google.com/github/zurutech/gans-from-theory-to-production/blob/master/2.%20GANs%20in%20Tensorflow/2.2.%20GANs%20with%20tf.estimator%20and%20tf.data.ipynb'>
    <img align="left" src='https://cdn-images-1.medium.com/max/800/1*ZpNn76K98snC9vDiIJ6Ldw.jpeg'></img>
</a>


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GANs-with-Estimator-API-and-tensorflow-datasets" data-toc-modified-id="GANs-with-Estimator-API-and-tensorflow-datasets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GANs with Estimator API and tensorflow-datasets</a></span><ul class="toc-item"><li><span><a href="#Estimator-API" data-toc-modified-id="Estimator-API-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Estimator API</a></span></li><li><span><a href="#Tensorflow-datasets-(tfds):-Celeb-A" data-toc-modified-id="Tensorflow-datasets-(tfds):-Celeb-A-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tensorflow-datasets (tfds): Celeb-A</a></span></li><li><span><a href="#DCGAN-Discriminator" data-toc-modified-id="DCGAN-Discriminator-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>DCGAN Discriminator</a></span></li></ul></li></ul></div>

## GANs with Estimator API and tensorflow-datasets

In this notebook, we're going to try to define a GAN and its input data pipeline using the new APIs `tf.estimator` and `tf.dataset`. Our aim is to build a **face generator** where each image is a $64\times64\times3$ tensor.

### Estimator API

Estimator have **a lot** of advantages, the official guide<sup>[1](#1)</sup> perfectly describes them:

- You can run Estimator-based models on a local host or on a distributed multi-server environment without changing your model. Furthermore, you can run Estimator-based models on CPUs, GPUs, or TPUs without recoding your model.
- Estimators simplify sharing implementations between model developers.
- You can develop a state of the art model with high-level intuitive code. In short, it is generally much easier to create models with Estimators than with the low-level TensorFlow APIs.
- Estimators are themselves built on tf.keras.layers, which simplifies customization.
- Estimators build the graph for you.
- Estimators provide a safe distributed training loop that controls how and when to:
    - build the graph
    - initialize variables
    - load data
    - handle exceptions
    - create checkpoint files and recover from failures
    - save summaries for TensorBoard

When writing an application with Estimators, **you must separate the data input pipeline from the model**. This separation simplifies experiments with different data sets.

![bella](images/dataset-estimator.jpg)

The estimator API enforces a programming style. Every method of the API accepets a `model_fn` (model function) that must implement a well-defined signature.

When working with `tf.estimator`, we have to follow the API specification. The `tf.estimator.Estimator.__init__` method requires as first parameter a `model_fn` function.

The model function `model_fn` implements the ML algorithm and its behaviour in different conditions (train/eval/predict) and **must** have the following signature:

```python
def model_fn(
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # An instance of tf.estimator.ModeKeys
   params):  # Additional configuration
```

and **must** return an instance of `tf.estimator.EstimatorSpec` that defines how the caller (the estimator) interacts with the model.


In order to correctly separate the data input pipeline from the model, let's introduce `tensorflow-datasets` (`tfds`): the collection of datasets ready to use, all built upon the `tf.data.Dataset` API.

### Tensorflow-datasets (tfds): Celeb-A

The `tf.data` API has been designed to write complex input pipelines in a very simple manner. It uses the **named pattern idiom** (also called **method chaining**) and its methods are inspired to the functional programming languages that applies transformations to lists.

The most imporant class is the `tf.data.Dataset` class that represents a sequence of elements: can apply transformation to this sequence of elements in order to create our dataset.

Every method of `tf.estimator` that requires input data (`train`, `evaluation`, `train_and_evalute`, `predict`) expects as first parameter an `input_fn` function. This function should construct and return one of the following

- A `tf.data.Dataset` object that once executed returns a tuple
- A tuple

The tuple can be the pair `(features, labels)` where features and labels can be batches. The cardinality of the tuple, however, is not constrained to be 2, it can be any: it depends on how we're going to use the return value of the `input_fn`.

Using `tensorflow-datasets` we don't have to bother about the download, preprocess and iterator generator: for the most common dataset everything is ready to use.

Downloading, having all the information, and a `tf.data.Dataset` object ready-to-use is just a single line.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from IPython import display

dataset_name = "celeb_a"
if dataset_name not in tfds.list_builders():
    raise ValueError(f"Something wrong with tfds: missing {dataset_name}")
dataset, info = tfds.load(dataset_name,split=tfds.Split.ALL, with_info=True)

print(info.features["image"])

Image(shape=(218, 178, 3), dtype=tf.uint8)


Since our goal is to generate $64\times64$ images and training on batches, we can use method chaining on the returned `dataset` object to generate an optimized training input pipeline that fits all our needs.

In [2]:
def convert_and_resize(features):
    image = tf.image.convert_image_dtype(features["image"], dtype=tf.float32)
    image = (image - 0.5) * 2
    image = tf.image.resize(image, size=(64, 64))
    features["image"] = image
    return features

batch_size = 32
dataset = dataset.map(convert_and_resize).batch(batch_size, drop_remainder=True).prefetch(1)

Downloading, post-processing and create an optimized input pipeline is just these few lines above!

Since we're working on images, we'll use an architecture created for this purpose: DCGAN <sup>[3](#3)</sup>.

### DCGAN Discriminator

The discriminator of DCGAN is common CNN architecture: a stack of convolutional layers that downsample the input image (without using pooling layers) followed by 2 fully connected layers.

The output layer in the disciminator definition has the **linear** activation function (for the same reasong explained in the previous notebook).

**<big>WARNING</big>**: the estimator API uses the Tensorflow 1.x API;
Therefore, when defining the `model_fn` we have to work as we are used to do in Tensorflow 1.x, describing a **computational graph**.
For this reason, is required to use the Tensorflow **compatibility** module: `tf.compat.v1`.


In [5]:
def discriminator_fn(features, labels, mode, params):
    """Build the Discriminator network.
    Args:
        features: a batch of images to classify, expected input shape (None, 64, 64 , 3)
        labels: a batch of labels
        mode: the tf.estimator.ModeKey
        params: a dict of optional parameters
    Returns:
            The tf.estimator.EstimatorSpec that descibes the desired behaviour
    """
    
    # Let's suppose that features is a batch of both, generated and real images
    training = mode = tf.estimator.EstimatorSpec.TRAIN
    # In every mode, define the model
    net = tf.keras.layers.Conv2D(filters=128, activation=tf.nn.leaky_relu)(features)
    net = tf.keras.layers.BatchNormalization()(net, training)
    net = tf.keras.layers.Conv2D(filters=256, activation=tf.nn.leaky_relu)(net)
    net = tf.keras.layers.BatchNormalization()(net, training)
    net = tf.keras.layers.Conv2D(filters=512, activation=tf.nn.leaky_relu)(net)
    net = tf.keras.layers.BatchNormalization()(net, training)
    net = tf.keras.layers.Flatten()(net)
    D = tf.keras.layers.Dense(1)(net)
    
    # Let's suppose that labels is a batch of labels where 1 is the real image
    # and 0 is the label associated to the generated image
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(y_true=labels, y_pred=D)
    
    if training:
        train_op = tf.compat.v1.train.AdamOptimizer(1e-5).minimize(loss)
        return tf.estimator.EstimatorSpec(
            mode, predictions=D, loss=loss, train_op=train_op)
    # in PREDICT or EVAL mode, just return the estimaor spec with the requested mode
    # and with the loss function (but NO the optimization step)
    return tf.estimator.EstimatorSpec(mode, loss=loss)

The discriminator has been correctly defined, **unfortunately** we had to suppose that the `features, labels` parameters contain what we do expect: both generated and real images.

*Is this really possibile when using the `tf.estimator` + `tf.data` API?*

Tecnically yes, but with a lot of struggle (that someone else at Google already did!):

- `tf.estimator.train` has been defined to train only one model at a time: how can we train both the generator and the discriminator using this function?
- The dataset we defined, that could be used in any classification problem, should be changed in order to add the noise vector required by the generator network -> `tf.data.Dataset` is no more an advantage and we have to change our code?
- How can we use two different `model_fn` (one for $G$ and one for $D$) and how can we connect the two models using a single estimator?

Maybe the simple estimator is not enough...
Lukily **an estimator tought to work with GANs has been introduced in the TFGAN library: GANEstimator**.

---
<a id="1">[1]</a>: https://www.tensorflow.org/guide/estimators

<a id="2">[2]</a>: For a more complete description of the `tf.data` API: https://www.tensorflow.org/guide/datasets

<a id="3">[3]</a>: Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks https://arxiv.org/pdf/1511.06434